# Agent Interaction Simulation

This notebook demonstrates how to use the Agent Simulator to create an interactive conversational experience with a wearable data coach. The agent can respond to user queries about their data and provide personalized recommendations.

In [ ]:
# Import necessary libraries
import sys
import os
import json
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Add the src directory to the path so we can import our modules
sys.path.append('../')
from src.llm_engine import LLMEngine
from src.agent_simulator import AgentSimulator

# Set up plotting
%matplotlib inline

## 1. Load Processed Data

First, let's load the processed data and insights from previous notebooks.

In [ ]:
# Load combined features
processed_dir = '../data/processed'
with open(os.path.join(processed_dir, 'combined_features.json'), 'r') as f:
    combined_features = json.load(f)

# Load user goals
with open(os.path.join(processed_dir, 'user_goals.json'), 'r') as f:
    user_goals = json.load(f)

# Get the most recent day's data
dates = list(combined_features.keys())
dates.sort()
latest_date = dates[-1]
latest_features = combined_features[latest_date]

print(f"Loaded data for {len(combined_features)} days")
print(f"User: {user_goals['name']}")
print(f"Latest data date: {latest_date}")

## 2. Initialize Agent Simulator

Now, let's initialize the Agent Simulator with our data.

In [ ]:
# Initialize LLM engine
llm_engine = LLMEngine()

# Initialize agent simulator with the latest features
agent = AgentSimulator(llm_engine, latest_features)

# Set user goals
user_goals_dict = {}
for goal in user_goals['primary_goals']:
    user_goals_dict[goal['area']] = goal['goal']
    
agent.set_user_goals(user_goals_dict)

# Set agent persona
agent.set_agent_persona({
    "name": "FitCoach",
    "role": "Wearable Data Coach",
    "tone": "coach",
    "expertise": ["running", "recovery", "sleep optimization", "stress management"]
})

# Check if API key is set
api_key_set = os.environ.get("OPENAI_API_KEY") is not None
if not api_key_set:
    print("WARNING: OpenAI API key not set. Agent responses will be simulated.")
    print("To set the API key, run: export OPENAI_API_KEY='your-api-key'")

## 3. Simulate a Conversation

Let's simulate a conversation with the agent to see how it responds to different queries.

In [ ]:
def simulate_conversation(initial_prompt, num_exchanges=3):
    """Simulate a conversation with the agent."""
    # Define follow-up questions based on the context
    follow_ups = [
        "Can you give me more specific recommendations?",
        "How does this compare to my previous data?",
        "What should I focus on improving first?",
        "What's the science behind this recommendation?",
        "How long until I should see improvements?"
    ]
    
    # Start with initial prompt
    print(f"User: {initial_prompt}")
    
    if api_key_set:
        response, _ = agent.process_message(initial_prompt)
    else:
        response = "[This is a simulated response. Set OPENAI_API_KEY to generate real responses.]"
        agent.conversation_history.append({
            "user": initial_prompt,
            "assistant": response,
            "timestamp": datetime.now().isoformat()
        })
    
    print(f"Assistant: {response}\n")
    
    # Simulate additional exchanges
    for i in range(num_exchanges - 1):
        if i < len(follow_ups):
            follow_up = follow_ups[i]
        else:
            follow_up = "Thank you for the information."
            
        print(f"User: {follow_up}")
        
        if api_key_set:
            response, _ = agent.process_message(follow_up)
        else:
            response = f"[This is a simulated response to '{follow_up}'. Set OPENAI_API_KEY to generate real responses.]"
            agent.conversation_history.append({
                "user": follow_up,
                "assistant": response,
                "timestamp": datetime.now().isoformat()
            })
        
        print(f"Assistant: {response}\n")
    
    return agent.conversation_history

In [ ]:
# Simulate a conversation about recovery
print("=== CONVERSATION ABOUT RECOVERY ===\n")
recovery_conversation = simulate_conversation(
    "How should I adjust my training based on my recovery score?",
    num_exchanges=2
)

In [ ]:
# Save the conversation
outputs_dir = '../outputs'
os.makedirs(outputs_dir, exist_ok=True)
agent.save_conversation(os.path.join(outputs_dir, 'recovery_conversation.json'))
print("Saved conversation to outputs/recovery_conversation.json")

In [ ]:
# Reset the agent for a new conversation
agent.conversation_history = []

# Simulate a conversation about sleep
print("=== CONVERSATION ABOUT SLEEP ===\n")
sleep_conversation = simulate_conversation(
    "I've been waking up tired despite sleeping 7 hours. What could be the issue?",
    num_exchanges=2
)

# Save the conversation
agent.save_conversation(os.path.join(outputs_dir, 'sleep_conversation.json'))
print("Saved conversation to outputs/sleep_conversation.json")

## 4. Interactive Agent Interface

Let's create an interactive interface for chatting with the agent.

In [ ]:
# Reset the agent for a new conversation
agent.conversation_history = []

# Create widgets for the chat interface
output = widgets.Output()
text_input = widgets.Text(description="You:", placeholder="Type your message here...")
send_button = widgets.Button(description="Send")
clear_button = widgets.Button(description="Clear Chat")
save_button = widgets.Button(description="Save Chat")

# Define callback functions
def on_send_button_clicked(b):
    user_input = text_input.value
    if not user_input.strip():
        return
    
    text_input.value = ""
    
    with output:
        print(f"You: {user_input}")
        
        if api_key_set:
            response, _ = agent.process_message(user_input)
        else:
            response = f"[This is a simulated response to '{user_input}'. Set OPENAI_API_KEY to generate real responses.]"
            agent.conversation_history.append({
                "user": user_input,
                "assistant": response,
                "timestamp": datetime.now().isoformat()
            })
        
        print(f"FitCoach: {response}\n")

def on_clear_button_clicked(b):
    agent.conversation_history = []
    with output:
        clear_output()
        print("Chat cleared. Start a new conversation!\n")

def on_save_button_clicked(b):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"conversation_{timestamp}.json"
    agent.save_conversation(os.path.join(outputs_dir, filename))
    with output:
        print(f"Conversation saved to outputs/{filename}\n")

# Connect callbacks to buttons
send_button.on_click(on_send_button_clicked)
clear_button.on_click(on_clear_button_clicked)
save_button.on_click(on_save_button_clicked)

# Handle Enter key in text input
def on_text_input_submitted(sender):
    on_send_button_clicked(None)
text_input.on_submit(on_text_input_submitted)

# Display the chat interface
display(widgets.VBox([
    widgets.HTML("<h3>Chat with FitCoach</h3>"),
    output,
    widgets.HBox([text_input, send_button]),
    widgets.HBox([clear_button, save_button])
]))

# Initialize the chat
with output:
    print("Welcome to FitCoach! I'm your wearable data coach. Ask me anything about your fitness, sleep, or recovery data.\n")

## 5. Suggested Conversation Starters

Here are some suggested questions to ask the agent:

In [ ]:
suggested_questions = [
    "What does my recovery score mean today?",
    "How should I adjust my training for my half marathon goal?",
    "What's the relationship between my HRV and sleep quality?",
    "I've been feeling more stressed lately. What does my data show?",
    "How can I improve my deep sleep percentage?",
    "Should I do a hard workout today based on my metrics?",
    "What trends do you see in my data over the past few days?",
    "How does my current training compare to what's optimal for my goals?"
]

for i, question in enumerate(suggested_questions, 1):
    print(f"{i}. {question}")

## 6. Advanced: Agent with Context-Aware Responses

Let's enhance our agent to provide more context-aware responses by incorporating historical data.

In [ ]:
class EnhancedAgent(AgentSimulator):
    """Enhanced agent with context-aware responses."""
    
    def __init__(self, llm_engine, feature_data=None, historical_data=None):
        """Initialize the enhanced agent."""
        super().__init__(llm_engine, feature_data)
        self.historical_data = historical_data or {}
    
    def load_historical_data(self, historical_data):
        """Load historical data for trend analysis."""
        self.historical_data = historical_data
    
    def _build_system_prompt(self):
        """Build an enhanced system prompt with historical context."""
        # Get basic system prompt
        system_prompt = super()._build_system_prompt()
        
        # Add historical context if available
        if self.historical_data:
            system_prompt += "\n\nYou have access to historical data for trend analysis. "
            system_prompt += "When relevant, refer to trends and patterns in the user's data over time."
        
        return system_prompt
    
    def _get_relevant_data(self, user_input):
        """Get data relevant to the user's query, including historical trends."""
        # Get basic relevant data
        relevant_data = super()._get_relevant_data(user_input)
        
        # Add historical trends if relevant
        if self.historical_data and any(term in user_input.lower() for term in ['trend', 'history', 'pattern', 'change', 'improve', 'progress']):
            # Add trend information
            trend_data = self._analyze_trends(user_input)
            if trend_data:
                relevant_data += "\n\n## HISTORICAL TRENDS\n" + trend_data
        
        return relevant_data
    
    def _analyze_trends(self, user_input):
        """Analyze trends in historical data relevant to the user's query."""
        if not self.historical_data:
            return ""
        
        trend_text = []
        
        # Determine which metrics to analyze based on the query
        metrics_to_analyze = []
        
        if any(term in user_input.lower() for term in ['sleep', 'rest', 'tired']):
            metrics_to_analyze.extend(['sleep_total_sleep_hours', 'sleep_deep_sleep_percentage', 'sleep_rem_sleep_percentage'])
        
        if any(term in user_input.lower() for term in ['recovery', 'readiness', 'strain']):
            metrics_to_analyze.extend(['recovery_score', 'hrv_rmssd_mean'])
        
        if any(term in user_input.lower() for term in ['activity', 'exercise', 'workout', 'training']):
            metrics_to_analyze.extend(['activity_total_steps', 'activity_active_minutes'])
        
        # If no specific metrics identified, use a default set
        if not metrics_to_analyze:
            metrics_to_analyze = ['recovery_score', 'hrv_rmssd_mean', 'sleep_total_sleep_hours']
        
        # Analyze each metric
        for metric in metrics_to_analyze:
            values = []
            dates = []
            
            for date, features in self.historical_data.items():
                if metric in features:
                    values.append(features[metric])
                    dates.append(date)
            
            if len(values) >= 2:
                # Sort by date
                sorted_data = sorted(zip(dates, values), key=lambda x: x[0])
                dates, values = zip(*sorted_data)
                
                # Calculate trend
                first_value = values[0]
                last_value = values[-1]
                change = last_value - first_value
                percent_change = (change / first_value) * 100 if first_value != 0 else 0
                
                # Format trend information
                metric_name = metric.replace('_', ' ').title()
                direction = "increased" if change > 0 else "decreased" if change < 0 else "remained stable"
                
                trend_text.append(f"- {metric_name} has {direction} by {abs(percent_change):.1f}% from {first_value:.1f} to {last_value:.1f} over the past {len(values)} days")
        
        return "\n".join(trend_text) if trend_text else ""

# Initialize enhanced agent
enhanced_agent = EnhancedAgent(llm_engine, latest_features, combined_features)

# Set user goals and persona
enhanced_agent.set_user_goals(user_goals_dict)
enhanced_agent.set_agent_persona({
    "name": "FitCoach Pro",
    "role": "Advanced Wearable Data Coach",
    "tone": "coach",
    "expertise": ["running", "recovery", "sleep optimization", "trend analysis", "stress management"]
})

# Simulate a conversation with the enhanced agent
print("=== CONVERSATION WITH ENHANCED AGENT ===\n")
enhanced_conversation = simulate_conversation(
    "What trends do you see in my recovery data over the past few days?",
    num_exchanges=2
)

# Save the conversation
enhanced_agent.save_conversation(os.path.join(outputs_dir, 'enhanced_conversation.json'))
print("Saved conversation to outputs/enhanced_conversation.json")

## Summary

In this notebook, we've demonstrated how to use the Agent Simulator to create an interactive conversational experience with a wearable data coach:

1. **Agent Initialization**: We initialized the agent with user data, goals, and a coaching persona.

2. **Simulated Conversations**: We simulated conversations about recovery and sleep to see how the agent responds to different queries.

3. **Interactive Interface**: We created an interactive chat interface for real-time conversations with the agent.

4. **Enhanced Agent**: We extended the agent to provide context-aware responses by incorporating historical data and trend analysis.

This conversational agent can be integrated into a mobile app, web dashboard, or other user interfaces to provide personalized coaching based on wearable data. The agent can help users understand their data, set appropriate goals, and make informed decisions about their training, recovery, and sleep.

Future enhancements could include:
- Integration with real-time data from wearable devices
- Personalized training plan generation
- Adaptive goal setting based on progress
- Multi-modal interactions (voice, images, etc.)
- Integration with calendar and scheduling tools